# FIT simulations

### Little bit of theory:

**Transfer entropy**: the mutual information between $Y_{pres}$ and $X_{past}$, conditioned on $Y_{past}$

$$ TE(X\rightarrow Y) = I(X_{past}; Y_{pres}|Y_{past}) $$

where

$$ I(S=s,X_i) = \sum_{x_i \in X_i} p(x_i|s)\,\log\frac{p(s|x_i)}{p(s)}$$

is the specific information that the source $X_i$ carries about a specific outcome of the target variable $s\in S$.

**FIT**:

$$ FIT = \text{min}[SUI(S: X_{past}, Y_{pres} \ Y_{past} ),\, SUI(Y_{pres}: X_{past}, S\ Y_{past})] $$

where 
- $SUI(S: X_{past}, Y_{pres} \ Y_{past} )$ is the information about $S$ that $X_{past}$ shares with $Y_{pres}$ and it's unique with respect to $Y_{past}$ and it's defined as the difference between the shared information that $X_{past}, Y_{pres}$ carry about $S$ and the shared information that $X_{past}, Y_{pres}$ and $Y_{past}$ carry about $S$
- $SUI(Y_{pres}: X_{past}, S\ Y_{past})$ is the information about $Y_{pres}$ that $X_{past}$ shares with $S$ and it's unique with respect to $Y_{past}$

Moreover

$$ SUI(S:X_1,X_2) = \sum_{s\in S} p(s) \text{min}_{X_i \in \{ X_1, X_2\} } I(S=s,X_i) $$

is the shared information that $X_1$ and $X_2$ carry about $S$.

**Sender activity**: 2D variable

$$X(t)_{stim} = S(t)(1+N(0, \sigma_{stim}))$$

where $S(t)$ is a step function equal to the value of the stimulus $s \in [1,4]$ during the time window $[200,250]\,ms$.

$$X(t)_{noise} = N(0,\sigma)$$

**Receiver activity**: 1D variable

$$Y(t) = W_{stim}X_{stim}(t-\delta) + W_{noise}X_{noise}(t-\delta) + N(0,\sigma)$$

where the **delay** $\delta$ is chosen randomly from a uniform distribution in $[40,60]\,ms$ in step of $10\,ms$, moreover $\sigma = 2$ and $\sigma_{stim} = \sigma/5 = 0.4$.

**Numerical computation of FIT**: discretization of neural activity into a number R of equipopulated bins and empirical computation of the occurence frequency of each binned response across all available trials.

FIT/TE are computed at the first time instant in which $Y$ received information from $X$.

Total of 50 simulation with 500 trials per stimulus each one.

### WHAT TO DO

- **Fig. 2A**: evaluation of FIT and TE for each value of $W_{noise}$ and $W_{stim}$
- **Fig. 2B**: FIT and TE as a function of time with $W_{stim}=0.5$ and $W_{noise}=1$ (FIT/TE values computed at all points and averaged over delays to obtain temporal profiles of transmitted information)

In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd

In [2]:
npr.seed(1)

Simulation parameters

In [3]:
nTrials_per_stim = 500      # number of trials for each simulation
simReps = 50                # number of simulations
nShuff = 10                 # number of permutations

w_sig = np.linspace(0, 1, num=11)      # signal weights for Y computation
w_noise = np.linspace(0, 1, num=11)    # noise weights for Y computation
stdX_noise = 2                      # std of gaussian noise in X_noise
stdY = 2                            # std of gaussian noise in Y
ratio = 0.2                         # ratio between stdX_sig and stdX_noise
stdX_sig = ratio * stdX_noise       # std of gaussian noise in X_signal

Global parameters

In [4]:
simLen = 60                             # simulation length in units of 10 ms
stimWin = [30, 35]                      # X stimulus encoding window in units of 10 ms
delays = np.linspace(4, 6, num=3, dtype=int)
n_binsS = 4                             # number of stimulus values
n_binsX = 3
n_binsY = 3
eps = 1e-52                             # infinitesimal value

nTrials = nTrials_per_stim * n_binsS

In [5]:
# Draw random delay for each simulation repetition
reps_delays = npr.choice(delays, simReps, replace=True)

Structures

In [6]:
fit = np.full((simReps, len(w_sig), len(w_noise)), np.nan)
di = fit.copy()
dfi = fit.copy()
fitsh = np.full((simReps, len(w_sig), len(w_noise), nShuff), np.nan)
dish = fitsh.copy()
dfish = fitsh.copy()
fitsh_cond = np.full((simReps, len(w_sig), len(w_noise), nShuff), np.nan)
dish_cond = fitsh_cond.copy()
dfish_cond = fitsh_cond.copy()

In [7]:
for simIdx in range(simReps):
    print('Simulation number: ', simIdx, '\n')
    for sigIdx in range(len(w_sig)):
        for noiseIdx in range(len(w_noise)):
            # draw the stimulus value for each trial
            S = npr.randint(1, n_binsS + 1, size=nTrials)

            # simulate neural activity
            X_noise = npr.normal(0, stdX_noise, size=(simLen, nTrials)) # Noise time series
        
            X_signal = eps * npr.normal(0, stdX_noise, size=(simLen, nTrials)) # Infinitesimal signal to avoid binning 

            X_signal[stimWin[0]:stimWin[1],:] = np.tile(S, (stimWin[1]-stimWin[0], 1)) # Assigning Stimulus to Window

            # Adding multiplicative noise (we changed the dimension from nTrials to (simLen, nTrials) to have a different error for each time step)
            X_signal = X_signal * (1 + npr.normal(0, stdX_sig, size=(simLen, nTrials))) 

            # Time lagged single-trial input from the 2 dimensions of X and Y (we multpily everything by the weights, they mutiply only the stim/noise)
            X2Ysig = w_sig[sigIdx] * np.vstack((eps * npr.normal(0, stdX_noise, size=(reps_delays[simIdx], nTrials)),\
                      X_signal[0:len(X_signal)-reps_delays[simIdx],:]))
            X2Ynoise = w_noise[noiseIdx] * np.vstack((eps * npr.normal(0, stdX_noise, size=(reps_delays[simIdx], nTrials)),\
                      X_noise[0:len(X_signal)-reps_delays[simIdx],:]))

            # Computing Y + gaussian noise
            Y = X2Ysig + X2Ynoise + npr.normal(0,stdY,size=(simLen, nTrials))

            # First time point at which Y receives stim info from X
            t = stimWin[0] + reps_delays[simIdx]
            d = reps_delays[simIdx]

            # Discretize Neural Activity
            _, bin_edges = pd.qcut(X_noise[t-d,:], n_binsX, retbins=True)
            bX_noise = np.digitize(X_noise[t-d, :], bins=bin_edges, right=False)

            _, bin_edges = pd.qcut(X_signal[t-d,:], n_binsX, retbins=True)
            bX_sig = np.digitize(X_signal[t-d,:], bins=bin_edges, right=False)

            _, bin_edges = pd.qcut(Y[t,:], n_binsY, retbins=True)
            bYt = np.digitize(Y[t,:], bins=bin_edges, right=False)

            _, bin_edges = pd.qcut(Y[t-d,:], n_binsY, retbins=True)
            bYpast = np.digitize(Y[t-d,:], bins=bin_edges, right=False)

            bx = (bX_sig - 1) * n_binsX + bX_noise


Simulation number:  0 

Simulation number:  1 

Simulation number:  2 

Simulation number:  3 

Simulation number:  4 

Simulation number:  5 

Simulation number:  6 

Simulation number:  7 

Simulation number:  8 

Simulation number:  9 

Simulation number:  10 

Simulation number:  11 

Simulation number:  12 

Simulation number:  13 

Simulation number:  14 

Simulation number:  15 

Simulation number:  16 

Simulation number:  17 

Simulation number:  18 

Simulation number:  19 

Simulation number:  20 

Simulation number:  21 

Simulation number:  22 

Simulation number:  23 

Simulation number:  24 

Simulation number:  25 

Simulation number:  26 

Simulation number:  27 

Simulation number:  28 

Simulation number:  29 

Simulation number:  30 

Simulation number:  31 

Simulation number:  32 

Simulation number:  33 

Simulation number:  34 

Simulation number:  35 

Simulation number:  36 

Simulation number:  37 

Simulation number:  38 

Simulation number:  39 

Simulation

In [11]:
S = npr.randint(1, n_binsS + 1, size=nTrials)
print(np.shape(S))

# simulate neural activity
X_noise = npr.normal(0, stdX_noise, size=(simLen, nTrials)) # Noise time series
        
X_signal = eps * npr.normal(0, stdX_noise, size=(simLen, nTrials)) # Infinitesimal signal to avoid binning 

X_signal[stimWin[0]:stimWin[1],:] = np.tile(S, (stimWin[1]-stimWin[0], 1)) # Assigning Stimulus to Window

# Adding multiplicative noise (we changed the dimension from nTrials to (simLen, nTrials) to have a different error for each time step)
X_signal = X_signal * (1 + npr.normal(0, stdX_sig, size=(simLen, nTrials))) 

#print(np.shape(eps * npr.normal(0, stdX_noise, size=(reps_delays[0], nTrials))))
#print(np.shape(w_sig[0] * X_signal[0:len(X_signal)-reps_delays[0],:]))

X2Ysig = np.vstack((eps * npr.normal(0, stdX_noise, size=(reps_delays[0], nTrials)),\
                     w_sig[1] * X_signal[0:len(X_signal)-reps_delays[0],:]))
print(X2Ysig)


(2000,)
[[-9.77442426e-54  3.58433197e-52  1.20897343e-52 ...  2.02674785e-52
  -1.36445505e-52 -2.53343960e-52]
 [-1.56807650e-52  2.18690700e-53 -1.29837535e-52 ... -1.43495403e-52
   4.49500021e-53  1.04878661e-52]
 [ 5.98277173e-53 -4.20367123e-52  2.61021670e-52 ... -1.49064672e-52
  -2.45033187e-53  2.94151298e-52]
 ...
 [-1.66801829e-53 -1.22987247e-53  4.98928151e-53 ...  4.59914380e-53
  -6.62644711e-53 -6.56691048e-53]
 [ 2.59025823e-53  1.51260117e-53 -1.31211531e-53 ... -2.66455396e-54
   4.31468110e-54 -7.74063844e-54]
 [ 6.97837815e-54 -6.34423549e-54  1.67402663e-54 ...  1.02051195e-53
   1.57734201e-53 -2.13863714e-54]]


In [9]:
len(X_signal)

60

In [10]:
np.shape(reps_delays)

(50,)